# Get_Data
## Ratios and Values Needed:
- **P/ Book Value**: Market Price per share, Book Value per share - YEEEEET
- **P / Sales**: Market cap / TTM Revenue or sales - YEEEET
- **EBITDA to EV**: EBITDA / EV
- **P / CF** : Stock Price (Avg price) / operating cash flow (TTM)
- **P / Earnings**: Market Price per share / Earnings per share
- **Shareholder Yield**

### API Routes
* API BASE: GET /stock/{symbol}/
* Price to Book: /advanced-stats
* Price to Sales: /advanced-stats
* EBITDA / EV: both in /advanced-stats
* Price to Cash Flow: Price:/chart/{20190220 (date)} CF:/cash-flow
* Price to Earnings Ratio: /advanced-stats
* Shareholder Yield: div yield in /advanced-stats

#### Problems:
* /advanced-stats is paid, also /cash-flow is paid

### Updated Routes using FCM:
* Route Base: https://financialmodelingprep.com/api/v3
* key-metrics: /company-key-metrics/AAPL?period=quarter
* Price: /historical-price-full/AAPL?serietype=line
* Price to Book: key-metrics
* Price to Sales: Market Cap: key-metrics / Revenue per share: key -metrics
* EBITDA to EV: Given as EV / EBITDA: key-metrics
* Price to Cash Flow: Price: price route / Cash Flow: key-metrics
* Price to Earnings: ratio in key-metrics
* Shareholder yield : Dividen Yield in key-metric (calc this)

In [1]:
import pandas as pd
import requests
import json 

stocks = ['AAPL' , 'MSFT', "F", "FIT", "TWTR", "AMZN", "ATVI", "MMM", "CVX", "UNP"]
req_attr = ["PB ratio" , 
            "Market Cap",
            "Revenue per Share", 
            "Enterprise Value over EBITDA",
            "Operating Cash Flow per Share",
            "PE ratio",
            "Dividend Yield",
           ]

In [2]:
def get_curr_price(ticker):
    response = requests.get("https://financialmodelingprep.com/api/v3/historical-price-full/{}?serietype=line".format(ticker.upper()))
    if response.status_code == 404:
        return "Stock Info not Available"
    result = response.json()
    return result["historical"][-1]["close"]    

In [3]:
def get_metrics(ticker):
    response = requests.get("https://financialmodelingprep.com/api/v3/company-key-metrics/{}?period=quarter".format(ticker))
    if response.status_code == 404:
        return "Stock Info not Available"
    comp_key_metrics = response.json()
    values_list = comp_key_metrics["metrics"][0]
    comp_info =  dict.fromkeys(req_attr, 0)
    for attr in req_attr:
        comp_info[attr] = values_list[attr]
    return comp_info

In [4]:
def build_company_dict(ticker):
    metrics_dict = get_metrics(ticker)
    curr_price = get_curr_price(ticker)
    metrics_dict["Market Price"] = curr_price
    return metrics_dict

In [5]:
def build_dataset(stocks):
    company_metrics_dict = build_company_dict('aapl')
    df = pd.DataFrame(index=stocks, columns=['PB ratio', 'Market Cap', 'Revenue per Share', 'Enterprise Value over EBITDA', 'Operating Cash Flow per Share', 'PE ratio', 'Dividend Yield', 'Market Price'])
    for ticker in stocks:
        df.loc[ticker] = build_company_dict(ticker)
    return df

In [83]:
import time
start_time = time.time()
df = build_dataset(stocks)
print("Took {}".format(time.time() - start_time))
df.head()

NameError: name 'build_dataset' is not defined

In [7]:
df.loc["F"].tolist()

['1.1202',
 '40493890895.15',
 '9.7523',
 '12.9059',
 '1.6222',
 '18.125',
 '0.0591',
 8.905]

In [173]:
df.to_csv("company_metrics.csv")

In [125]:
import pandas as pd
import numpy as np

df = pd.read_csv("ratios.csv", index_col=0)

In [126]:
df

,pb,pe,ps,e_ev,pcf,dy
AAPL,9.5506,17.0372,11.7728,12.0424,92.434598,0.0147
MSFT,10.1498,26.5519,4.4046,17.7757,67.279726,0.0133
F,1.1202,18.1250,9.7523,12.9059,5.591172,0.0591
FIT,1.8018,20.2959,1.2241,14.8700,67.279726,0.0196
TWTR,3.3238,11.6764,1.0945,22.7005,91.358585,0.0196
AMZN,17.8594,77.9477,128.6085,28.6593,95.560398,0.0196
ATVI,3.0379,21.5909,1.8225,15.2035,276.417910,0.0078
MMM,9.9985,20.6521,14.1440,14.8700,56.651257,0.0321
CVX,1.5081,16.0463,20.5636,6.4380,25.018822,0.0370
UNP,6.6586,20.2959,7.9320,13.2221,59.297761,0.0196


In [127]:
def rank_ratio():
    df['pb_rank'] = df['pb'].rank(pct=True, ascending=True) * 100
    df['pe_rank'] = df['pe'].rank(pct=True, ascending=True) * 100
    df['ps_rank'] = df['ps'].rank(pct=True, ascending=True) * 100
    df['e_ev_rank'] = df['e_ev'].rank(pct=True, ascending=True) * 100
    df['pcf_rank'] = df['pcf'].rank(pct=True, ascending=True) * 100
    df['dy_rank'] = df['dy'].rank(pct=True, ascending=False) * 100

In [128]:
ranked = rank_ratio()

In [129]:
df

,pb,pe,ps,e_ev,pcf,dy,pb_rank,pe_rank,ps_rank,e_ev_rank,pcf_rank,dy_rank
AAPL,9.5506,17.0372,11.7728,12.0424,92.434598,0.0147,70.0,30.0,70.0,20.0,80.0,80.0
MSFT,10.1498,26.5519,4.4046,17.7757,67.279726,0.0133,90.0,90.0,40.0,80.0,55.0,90.0
F,1.1202,18.1250,9.7523,12.9059,5.591172,0.0591,10.0,40.0,60.0,30.0,10.0,10.0
FIT,1.8018,20.2959,1.2241,14.8700,67.279726,0.0196,30.0,55.0,20.0,55.0,55.0,55.0
TWTR,3.3238,11.6764,1.0945,22.7005,91.358585,0.0196,50.0,10.0,10.0,90.0,70.0,55.0
AMZN,17.8594,77.9477,128.6085,28.6593,95.560398,0.0196,100.0,100.0,100.0,100.0,90.0,55.0
ATVI,3.0379,21.5909,1.8225,15.2035,276.417910,0.0078,40.0,80.0,30.0,70.0,100.0,100.0
MMM,9.9985,20.6521,14.1440,14.8700,56.651257,0.0321,80.0,70.0,80.0,55.0,30.0,30.0
CVX,1.5081,16.0463,20.5636,6.4380,25.018822,0.0370,20.0,20.0,90.0,10.0,20.0,20.0
UNP,6.6586,20.2959,7.9320,13.2221,59.297761,0.0196,60.0,55.0,50.0,40.0,40.0,55.0


In [133]:
def rank_ticker():
    #TODO: handle 0s, rank starting from 1
    rank_ratio()
    #df['ratio_avg'] = df.loc[:, 'pb_rank':'dy_rank'].mean(axis=1, numeric_only=True)
    df["ratios_total"] = df.loc[:, "pb_rank":"dy_rank"].sum(axis=1)
    df['VC2 Score'] = df['ratios_total'].rank(pct=True, ascending=True) * 100

In [134]:
rank_ticker()

In [135]:
df

,pb,pe,ps,e_ev,pcf,dy,pb_rank,pe_rank,ps_rank,e_ev_rank,pcf_rank,dy_rank,ratios_total,VC2 Score
AAPL,9.5506,17.0372,11.7728,12.0424,92.434598,0.0147,70.0,30.0,70.0,20.0,80.0,80.0,350.0,70.0
MSFT,10.1498,26.5519,4.4046,17.7757,67.279726,0.0133,90.0,90.0,40.0,80.0,55.0,90.0,445.0,90.0
F,1.1202,18.1250,9.7523,12.9059,5.591172,0.0591,10.0,40.0,60.0,30.0,10.0,10.0,160.0,10.0
FIT,1.8018,20.2959,1.2241,14.8700,67.279726,0.0196,30.0,55.0,20.0,55.0,55.0,55.0,270.0,30.0
TWTR,3.3238,11.6764,1.0945,22.7005,91.358585,0.0196,50.0,10.0,10.0,90.0,70.0,55.0,285.0,40.0
AMZN,17.8594,77.9477,128.6085,28.6593,95.560398,0.0196,100.0,100.0,100.0,100.0,90.0,55.0,545.0,100.0
ATVI,3.0379,21.5909,1.8225,15.2035,276.417910,0.0078,40.0,80.0,30.0,70.0,100.0,100.0,420.0,80.0
MMM,9.9985,20.6521,14.1440,14.8700,56.651257,0.0321,80.0,70.0,80.0,55.0,30.0,30.0,345.0,60.0
CVX,1.5081,16.0463,20.5636,6.4380,25.018822,0.0370,20.0,20.0,90.0,10.0,20.0,20.0,180.0,20.0
UNP,6.6586,20.2959,7.9320,13.2221,59.297761,0.0196,60.0,55.0,50.0,40.0,40.0,55.0,300.0,50.0


In [80]:
result_df = df.loc[:, "pb":"dy"]
result_df["VC2_Score"] = df["VC2 Score"]

In [81]:
result_df.sort_values(by = "VC2_Score", axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')